In [1]:
import pandas as p

trips = p.read_csv("data_2017_withweather.csv").filter(items = ["month","day","hour","TaxiID","TripSeconds","Fare","Company","PickupCommunityArea","temp_hour","rain_hour","temp_sdday","rain_sdday"])

In [2]:
def get_temp_cat(temp):
    if temp <= 0.6:
        return 0
    elif temp <= 6.4:
        return 1
    elif temp <= 12.5:
        return 2
    else:
        return 3

def get_temp_sd_cat(temp_sd):
    if temp_sd <= 1.5:
        return 0
    elif temp_sd <= 2.6:
        return 1
    elif temp_sd <= 3.8:
        return 2
    else:
        return 3
    
def get_rain_sd_cat(rain_sd):
    if rain_sd <= 0:
        return 0
    elif rain_sd <= 0.03:
        return 1
    elif rain_sd <= 0.42:
        return 2
    else:
        return 3

In [3]:
areas = [
    [1,2,3,4,9,10,11,12,13,14,76,77],        #Far North
    [15,16,17,18,19,20],                     #Northwest
    [5,6,7,21,22],                           #North
    [23,24,25,26,27,28,29,30,31],            #West
    [8,32,33],                               #Central
    [56,57,58,59,61,62,63,64,65,66,67,68],   #Southwest
    [34,35,36,37,38,39,40,41,42,43,60,69],   #South
    [70,71,72,73,74,75],                     #Far Southwest
    [44,45,46,47,48,49,50,51,52,53,54,55]    #Far Southeast
]

def get_side(area):
    for sidei in range(0,len(areas)):
        if area in areas[sidei]:
            return sidei
    return len(areas)                       #dummy that corresponds to unknown region

In [4]:
#encode trip data (temp, time,...) to categories used in analysis

import datetime
import math

dow = []
tod = []
weekno = []
temp_cat = []
rain = []
temp_sd_cat = []
rain_sd_cat = []
side = []
for index,row in trips.iterrows():
    date = datetime.datetime(2017,row["month"],row["day"])
    dow.append(date.weekday())
    weekno.append(date.isocalendar()[1])
    tod.append(math.floor(row["hour"]/6))
    temp_cat.append(get_temp_cat(row["temp_hour"]))
    rain.append(row["rain_hour"] > 0)
    temp_sd_cat.append(get_temp_sd_cat(row["temp_sdday"]))
    rain_sd_cat.append(get_rain_sd_cat(row["rain_sdday"]))
    side.append(get_side(row["PickupCommunityArea"]))
    
processed = p.DataFrame(data = {"dow":dow, "tod":tod, "weekno":weekno, "temp_cat":temp_cat, "rain":rain, "temp_sd_cat":temp_sd_cat, "rain_sd_cat":rain_sd_cat, "side":side})
trips_processed = trips.join(processed)

In [8]:
#create hash map for aggregated trip data

import numpy as np

taxiperweek = []
taxis = trips.loc[:,"TaxiID"].unique()
n_taxis = len(taxis)
for taxi_i in range(0,n_taxis):
    for week in range(0,52):
        taxiperweek.append({"sampleId":52*taxi_i+week,"company":None,"revenue":0,"rides_time":np.zeros((7,4)),"rides_rain":np.zeros(2),"rides_temp":np.zeros(4),"temp_sd_cat":np.zeros(4),"rain_sd_cat":np.zeros(4),"seconds_total": 0, "rate":0, "side":np.zeros(len(areas)+1)})

In [9]:
def get_taxiperweek_index(taxi_id,weekno):
    return np.where(taxis == taxi_id)[0][0]*52+weekno-1

In [10]:
#iterate through trips and update hash map

for index,row in trips_processed.iterrows():
    taxi_id = row["TaxiID"]
    if p.isnull(taxi_id):
        continue
    record = taxiperweek[get_taxiperweek_index(taxi_id,row["weekno"])]
    record["revenue"] += float(row["Fare"])
    record["rides_time"][row["dow"]][row["tod"]] += 1
    record["rides_rain"][int(row["rain"])] += 1
    record["rides_temp"][row["temp_cat"]] += 1
    record["temp_sd_cat"][row["temp_sd_cat"]] += 1
    record["rain_sd_cat"][row["rain_sd_cat"]] += 1
    record["seconds_total"] += float(row["TripSeconds"])
    seconds_total = record["seconds_total"]
    record["side"][row["side"]] += 1
    if seconds_total > 0:
        record["rate"] = record["revenue"] / seconds_total
    company = row["Company"]
    if company not in [None,""]:
        record["company"] = company

In [21]:
weekday_names = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
tod_names = ["Night","Morning","Afternoon","Evening"]
def decode_time(time_id):
    weekday = int(time_id / 7)
    tod = time_id % 4
    return weekday_names[weekday] + tod_names[tod]

side_names = ["Far North","Northwest","North","West","Central","Southwest","South","Far Southwest","Far Southeast","Unknown"]
def decode_side(side_id):
    return side_names[side_id]

In [26]:
#flatten data so that it can be printed to a csv with meaningful labels

taxiperweek_formatted = [["sampleId","company","revenue","rate","norain","rain"]]
for i in range(0,28):
    taxiperweek_formatted[0].append(decode_time(i))
for i in range(0,4):
    taxiperweek_formatted[0].append("temp" + str(i))
for i in range(0,4):
    taxiperweek_formatted[0].append("temp_sd" + str(i))
for i in range(0,4):
    taxiperweek_formatted[0].append("rain_sd" + str(i))
for i in range(0,len(areas) + 1):
    taxiperweek_formatted[0].append("side" + str(i))
    
for row in taxiperweek:
    row_formatted = []
    row_formatted.append(row["sampleId"])
    row_formatted.append(row["company"])
    row_formatted.append(row["revenue"])
    row_formatted.append(row["rate"])
    row_formatted += list(row["rides_rain"]) + list(row["rides_time"].flatten()) + list(row["rides_temp"]) + list(row["temp_sd_cat"]) + list(row["rain_sd_cat"]) + list(row["side"])
    taxiperweek_formatted.append(row_formatted)

In [27]:
weekly_stats = p.DataFrame(data = taxiperweek_formatted[1:], columns = taxiperweek_formatted[0])

In [28]:
weekly_stats.to_csv("weekly_stats.csv")

In [29]:
taxiperweek_cleaned = []
for row in taxiperweek_formatted[1:]:
    if row[2] != 0:
        taxiperweek_cleaned.append(row)
p.DataFrame(data = taxiperweek_cleaned, columns = taxiperweek_formatted[0]).to_csv("weekly_stats_nonzero.csv")